In [ ]:
!pip install transformers identitychain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 22.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for identitychain: filename=identitychain-0.1.0-py3-none-any.whl size=11851 sha256=d2549ddb09430871d6eb754500a9472858bc0f63d08adbc9a64104eb725c93ed
  Stored in directory: /root/.cache/pip/wheels/85/ba/35/2053d23dc06bed2df774662444f706b5cbc82e12815f680e78
Successfully built identitychain


In [ ]:
import os

# Set paths in the Colab environment
os.environ['HF_HOME'] = '/content/huggingface'
os.environ['IDENTITY_CHAIN_HOME'] = '/content/IdentityChain'
!mkdir -p /content/huggingface  # Create the Hugging Face model cache directory
!mkdir -p /content/IdentityChain/data

In [ ]:
MODEL = "semcoder/semcoder_s"  # SemCoder S model from Hugging Face
DATASET = "EvalPlus-Mini-v0.1.6_reformatted.jsonl"
TEMP = 0  # Temperature for generation; set to 0 for deterministic output

In [ ]:
MODEL_NAME = "semcoder_s"
OUTPUT_DIR = f"{os.environ['IDENTITY_CHAIN_HOME']}/tmp/{MODEL_NAME}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("semcoder/semcoder_s")
# model = AutoModelForCausalLM.from_pretrained("semcoder/semcoder_s")

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "semcoder/semcoder_s"

try:
    # Test loading the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("Tokenizer loaded successfully.")

    # Test loading the model
    model = AutoModelForCausalLM.from_pretrained(model_name)
    print("Model loaded successfully.")

except Exception as e:
    print("Error loading model or tokenizer:", e)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.26k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Tokenizer loaded successfully.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

Model loaded successfully.


In [ ]:
!rm -rf /content/IdentityChain
!git clone https://github.com/marcusm117/IdentityChain.git /content/IdentityChain  # Clone the repository

Cloning into '/content/IdentityChain'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 106 (delta 31), reused 67 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (106/106), 1.70 MiB | 54.45 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
# !conda create -n idchain python=3.10
# !conda activate idchain
# !pip uninstall fsspec
# !pip install fsspec==2024.10.0

!cd IdentityChain/ && pip install -e .

Obtaining file:///content/IdentityChain
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for identitychain (pyproject.toml) ... done
  Created wheel for identitychain: filename=identitychain-0.1.0-0.editable-py3-none-any.whl size=13567 sha256=668e4aa02ae88960e6219acefc048e1846ad78b02d68d5c51e951b67638e052d
  Stored in directory: /tmp/pip-ephem-wheel-cache-fg2wvxul/wheels/a5/a5/9c/0eda9cba365839b143caa5761206b8c6b828795d6a4ed0bf22
Successfully built identitychain


In [ ]:
import json

input_path = "/content/IdentityChain/data/HumanEval/HumanEvalPlus-Mini-v0.1.6.jsonl"
output_path = "/content/IdentityChain/data/HumanEval/HumanEvalPlus-Mini-v0.1.6_modified.jsonl"

with open(input_path, "r") as infile, open(output_path, "w") as outfile:
    for line in infile:
        data = json.loads(line)
        # Copy 'prompt' content to 'problem' if it doesn't already exist
        if "prompt" in data and "problem" not in data:
            data["problem"] = data["prompt"]
        outfile.write(json.dumps(data) + "\n")


In [ ]:
import json

input_path = "/content/IdentityChain/data/HumanEval/HumanEvalPlus-Mini-v0.1.6_modified.jsonl"

# Create a temporary list to store modified entries
modified_entries = []

# Read and modify the file in place
with open(input_path, "r") as infile:
    for line in infile:
        data = json.loads(line)
        # Add 'function_name' by copying from 'entry_point' if it exists
        if "entry_point" in data and "function_name" not in data:
            data["function_name"] = data["entry_point"]
        modified_entries.append(data)

# Overwrite the original file with modified entries
with open(input_path, "w") as outfile:
    for entry in modified_entries:
        outfile.write(json.dumps(entry) + "\n")

print("Updated dataset with 'function_name' added to each entry.")


Updated dataset with 'function_name' added to each entry.


In [ ]:
import json
import re

input_path = "/content/IdentityChain/data/HumanEval/HumanEvalPlus-Mini-v0.1.6_modified.jsonl"

# Read, modify, and overwrite each entry in the file
modified_entries = []

with open(input_path, "r") as infile:
    for line in infile:
        data = json.loads(line)

        # Add 'function_name' from 'entry_point' if it doesn't exist
        if "entry_point" in data and "function_name" not in data:
            data["function_name"] = data["entry_point"]

        # Extract 'function_signature' from the 'prompt'
        if "prompt" in data and "function_signature" not in data:
            match = re.search(r"def [\w_]+\([^)]*\) -> [\w\[\]]+:", data["prompt"])
            if match:
                data["function_signature"] = match.group(0)
            else:
                print(f"Warning: Unable to extract function signature for task_id {data['task_id']}")

        modified_entries.append(data)

# Overwrite the file with the modified entries
with open(input_path, "w") as outfile:
    for entry in modified_entries:
        outfile.write(json.dumps(entry) + "\n")

print("Dataset updated with 'function_signature' added.")


Dataset updated with 'function_signature' added.


In [ ]:
import subprocess

# Path to the modified dataset
modified_file_path = "/content/IdentityChain/data/HumanEval/HumanEvalPlus-Mini-v0.1.6_modified.jsonl"
gzipped_file_path = modified_file_path + ".gz"

# Compress the modified file
!gzip -c {modified_file_path} > {gzipped_file_path}

# Confirm the file was compressed
print(f"Compressed file created at: {gzipped_file_path}")

Compressed file created at: /content/IdentityChain/data/HumanEval/HumanEvalPlus-Mini-v0.1.6_modified.jsonl.gz


In [ ]:
#!gzip -d /content/IdentityChain/data/HumanEval/HumanEvalPlus-Mini-v0.1.6.jsonl.gz
# os.environ['TF_XLA_FLAGS'] = ''
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import subprocess

# Define the command with the correct path to the IdentityChain script
command = [
    "python", "/content/IdentityChain/examples/run_identity_chain_huggingface.py",
    "--model_name_or_path", "deepseek-ai/deepseek-coder-1.3b-instruct",
    "--hf_dir", "/content/huggingface",
    "--input_path", "/content/IdentityChain/data/HumanEval/HumanEvalPlus-Mini-v0.1.6_modified.jsonl",  # Update to the correct dataset path if needed
    "--output_dir", "/content/IdentityChain/tmp/semcoder_s",
    "--seq_length", "1536",
    "--gen_len", "512",
    "--greedy_early_stop",
    "--chain_length", "5",
    "--use_fp16",
    "--mask_func_name",
    "--bootstrap_method", "problem",
    "--resume_task_bs", "1",
    "--resume_task_run", "0",
    "--temperature", "0"
]

# Run the command and capture output
result = subprocess.run(command, capture_output=True, text=True)

# Print standard output and error messages to diagnose any issues
print("Standard Output:\n", result.stdout)
print("Standard Error:\n", result.stderr)

Standard Output:
 Loading model...
**********************************
****** Using fp16 precision ******
**********************************
--------- Prompt Configuration -----------
[{'role': 'system', 'content': "Solve a coding problem in Python. Given the function signature and the problem description in the docstring, you only need to continue to complete the function body. Please strictly follow the format of the example below! Don't write down any thought processes! Don't copy the problem description! You must use correct indentation! Make sure your return statement is always inside the function! Make sure your output always starts with an indentation of exactly 4 spaces! Output an indentation of 4 spaces first before you write anything else! You’d better be sure. \n\n"}, {'role': 'user', 'content': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """Check if in given list of numbers, are any two numbers closer to each oth